In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
cle = pd.read_csv('../TrainTestData/cle_metadata.csv')
vir = pd.read_csv('../TrainTestData/vir_metadata.csv')
hun = pd.read_csv('../TrainTestData/hun_metadata.csv')
swi = pd.read_csv('../TrainTestData/swi_metadata.csv')

In [3]:
data = pd.concat([cle,vir,hun,swi])
data

,0,1,2,3,4,5,6,7,8,9,...,7.3,8.3,9.3,10.3,11.3,12.3,13.3,14.3,15.3,num
0,1.099659,0.0,0.0,0.872684,1.073222,0.0,1.486944,0.0,2.189892,0.0,...,0.0,1.457568,0.0,0.0,0.589020,0.882712,0.0,0.0,1.598649,0.00
1,1.849904,0.0,0.0,0.620106,0.811350,0.0,1.943316,0.0,2.594235,0.0,...,0.0,1.488225,0.0,0.0,0.637975,0.901482,0.0,0.0,1.643992,0.00
2,2.224025,0.0,0.0,0.119797,0.067982,0.0,2.657312,0.0,3.285271,0.0,...,0.0,1.199812,0.0,0.0,0.546282,0.756742,0.0,0.0,1.321362,0.00
3,1.738161,0.0,0.0,0.520788,0.593032,0.0,2.272287,0.0,2.677983,0.0,...,0.0,1.394613,0.0,0.0,0.627924,0.883689,0.0,0.0,1.631824,0.00
4,1.402882,0.0,0.0,0.570546,0.819004,0.0,1.494263,0.0,2.169592,0.0,...,0.0,1.280468,0.0,0.0,0.586185,0.768512,0.0,0.0,1.481180,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0.352982,0.0,0.0,1.839385,2.316214,0.0,0.474900,0.0,1.077897,0.0,...,0.0,1.968976,0.0,0.0,0.851110,1.141455,0.0,0.0,2.026502,0.75
119,0.655624,0.0,0.0,1.390551,1.878792,0.0,0.452024,0.0,1.255075,0.0,...,0.0,1.707302,0.0,0.0,0.670807,1.002500,0.0,0.0,1.859282,0.50
120,0.161753,0.0,0.0,1.826201,2.245803,0.0,0.137622,0.0,0.954996,0.0,...,0.0,1.802060,0.0,0.0,0.740325,1.043203,0.0,0.0,1.887837,1.00
121,0.102779,0.0,0.0,1.974692,2.487005,0.0,0.091805,0.0,0.745667,0.0,...,0.0,1.693194,0.0,0.0,0.653663,1.047537,0.0,0.0,1.765075,0.25


In [4]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Y_train_binary = Y_train.apply(lambda x: 1 if x > 0 else 0)
Y_test_binary = Y_test.apply(lambda x: 1 if x > 0 else 0)

In [5]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# define the model architecture
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(64,1)))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.fit(X_train, Y_train_binary, epochs=1000, batch_size=32,callbacks=[callback])
Y_pred = model.predict(X_test).argmax(axis=1)
    
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred, digits=4))

Epoch 1/1000
20/20 [==============================] - 3s 102ms/step - loss: 0.4564 - accuracy: 0.8019
Epoch 2/1000
20/20 [==============================] - 2s 113ms/step - loss: 0.3690 - accuracy: 0.8588
Epoch 3/1000
20/20 [==============================] - 2s 109ms/step - loss: 0.3585 - accuracy: 0.8653
Epoch 4/1000
20/20 [==============================] - 3s 132ms/step - loss: 0.3566 - accuracy: 0.8604
Epoch 5/1000
20/20 [==============================] - 3s 142ms/step - loss: 0.3389 - accuracy: 0.8701
Epoch 6/1000
20/20 [==============================] - 3s 162ms/step - loss: 0.3357 - accuracy: 0.8734
Epoch 7/1000
20/20 [==============================] - 3s 153ms/step - loss: 0.3314 - accuracy: 0.8718
Epoch 8/1000
20/20 [==============================] - 3s 169ms/step - loss: 0.3322 - accuracy: 0.8734
Epoch 9/1000
20/20 [==============================] - 3s 151ms/step - loss: 0.3247 - accuracy: 0.8718
Epoch 10/1000
20/20 [==============================] - 3s 158ms/step - loss: 0.328

In [6]:
model.save('../Models/Meta_Only/Meta_CNN.h5')

# Test on Each Data Set

In [12]:
def Test(path_train,path_test,model_name):
    Train = pd.read_csv(path_train)
    Test = pd.read_csv(path_test)
    
    X_train = Train.iloc[:,:-1]
    Y_train = Train.iloc[:,-1]

    X_test = Test.iloc[:,:-1]
    Y_test = Test.iloc[:,-1]

    #binarize the target
    Y_train_binary = Y_train.apply(lambda x: 1 if x > 0 else 0)
    Y_test_binary = Y_test.apply(lambda x: 1 if x > 0 else 0)

    model = tf.keras.models.load_model(model_name)

    Y_pred = model.predict(X_test).argmax(axis=1)
    
    cm = confusion_matrix(Y_pred, Y_test_binary)
    print(cm)
    print(classification_report(Y_test_binary, Y_pred, digits=4))

# Cleveland

In [13]:
path_train = '../TrainTestData/cle_metadata_train.csv'
path_test = '../TrainTestData/cle_metadata_test.csv'
model = '../Models/Meta_Only/Meta_CNN.h5'
Test(path_train,path_test,model)

[[44  4]
 [ 4 39]]
              precision    recall  f1-score   support

           0     0.9167    0.9167    0.9167        48
           1     0.9070    0.9070    0.9070        43

    accuracy                         0.9121        91
   macro avg     0.9118    0.9118    0.9118        91
weighted avg     0.9121    0.9121    0.9121        91



# Virginia

In [14]:
path_train = '../TrainTestData/vir_metadata_train.csv'
path_test = '../TrainTestData/vir_metadata_test.csv'
model = '../Models/Meta_Only/Meta_CNN.h5'
Test(path_train,path_test,model)

[[ 9  3]
 [ 6 42]]
              precision    recall  f1-score   support

           0     0.7500    0.6000    0.6667        15
           1     0.8750    0.9333    0.9032        45

    accuracy                         0.8500        60
   macro avg     0.8125    0.7667    0.7849        60
weighted avg     0.8438    0.8500    0.8441        60



# Hungarian

In [15]:
path_train = '../TrainTestData/hun_metadata_train.csv'
path_test = '../TrainTestData/hun_metadata_test.csv'
model = '../Models/Meta_Only/Meta_CNN.h5'
Test(path_train,path_test,model)

[[49  3]
 [ 6 31]]
              precision    recall  f1-score   support

           0     0.9423    0.8909    0.9159        55
           1     0.8378    0.9118    0.8732        34

    accuracy                         0.8989        89
   macro avg     0.8901    0.9013    0.8946        89
weighted avg     0.9024    0.8989    0.8996        89



# Switzerland

In [16]:
path_train = '../TrainTestData/swi_metadata_train.csv'
path_test = '../TrainTestData/swi_metadata_test.csv'
model = '../Models/Meta_Only/Meta_CNN.h5'
Test(path_train,path_test,model)

[[ 2  4]
 [ 1 30]]
              precision    recall  f1-score   support

           0     0.3333    0.6667    0.4444         3
           1     0.9677    0.8824    0.9231        34

    accuracy                         0.8649        37
   macro avg     0.6505    0.7745    0.6838        37
weighted avg     0.9163    0.8649    0.8843        37

